<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_Part2_Chapter07_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
train = pd.read_csv('score_train.csv')
test = pd.read_csv('score_test.csv')

#EDA

In [31]:
train.shape, test.shape

((4198, 21), (1499, 20))

In [6]:
train.head(3)

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Mix,Payment_Behaviour,...,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month,Credit_Score
0,26.0,10.0,5.0,30.358905,200.0,NM,192.341800,321.431503,Good,Low_spent_Small_value_payments,...,36963.89,0.0,4.0,1.0,2.0,2902.324167,6.47,1195.96,46.459114,Standard
1,6.0,17.0,4.0,24.589796,70.0,Yes,53.110239,427.404738,Standard,High_spent_Medium_value_payments,...,34599.80,8.0,4.0,16.0,3.0,3130.316667,16.53,851.37,82.516689,Standard
2,29.0,16.0,6.0,39.456866,14.0,Yes,150.829491,239.039985,Bad,Low_spent_Small_value_payments,...,19531.18,7.0,10.0,25.0,5.0,1679.598333,20.74,3047.42,68.090357,Poor


In [7]:
test.head(3)

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Mix,Payment_Behaviour,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
0,19.0,20.0,11.0,28.893968,57.0,Yes,126.895309,217.045508,Bad,Low_spent_Small_value_payments,21.0,16644.79,7.0,6.0,24.0,9.0,1547.065833,9.93,3519.49,100.765767
1,6.0,6.0,7.0,34.443437,366.0,No,217.005753,572.710742,Good,Low_spent_Medium_value_payments,22.0,83358.30,5.0,3.0,5.0,3.0,6896.525000,2.66,727.88,169.936005
2,30.0,19.0,8.0,21.970573,210.0,Yes,382.538630,238.883672,Standard,High_spent_Medium_value_payments,33.0,66691.23,8.0,6.0,20.0,4.0,5553.602500,19.78,760.71,213.937948


In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       1499 non-null   float64
 1   Num_of_Delayed_Payment    1499 non-null   float64
 2   Num_Credit_Inquiries      1499 non-null   float64
 3   Credit_Utilization_Ratio  1499 non-null   float64
 4   Credit_History_Age        1499 non-null   float64
 5   Payment_of_Min_Amount     1499 non-null   object 
 6   Amount_invested_monthly   1499 non-null   float64
 7   Monthly_Balance           1499 non-null   float64
 8   Credit_Mix                1499 non-null   object 
 9   Payment_Behaviour         1499 non-null   object 
 10  Age                       1499 non-null   float64
 11  Annual_Income             1499 non-null   float64
 12  Num_Bank_Accounts         1499 non-null   float64
 13  Num_Credit_Card           1499 non-null   float64
 14  Interest

In [12]:
for col in train.select_dtypes(include='object').columns:
  print(col, ':', train[col].unique())


Payment_of_Min_Amount : ['NM' 'Yes' 'No']
Credit_Mix : ['Good' 'Standard' 'Bad']
Payment_Behaviour : ['Low_spent_Small_value_payments' 'High_spent_Medium_value_payments'
 'High_spent_Large_value_payments' 'High_spent_Small_value_payments'
 'Low_spent_Large_value_payments' 'Low_spent_Medium_value_payments']
Credit_Score : ['Standard' 'Poor' 'Good']


In [40]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(np.int64(0), np.int64(0))

#데이터 전처리

In [54]:
target = train.pop('Credit_Score')
print(train.shape, test.shape, target.shape)

(4198, 20) (1499, 20) (4198,)


In [55]:
#num : 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

cols = train.select_dtypes(exclude = 'object').columns
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

In [60]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [61]:
#obj : 레이블인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in train.select_dtypes(include='object').columns:
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])


#모델학습

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.3, random_state = 99)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(2938, 29) (1260, 29) (2938,) (1260,)


In [63]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

from sklearn.metrics import f1_score
y_pred = rfc.predict(X_val)
score = f1_score(y_val, y_pred, average = 'macro')
print(score)

0.6609398226024257


#결과

In [64]:
result = rfc.predict(test)
result_df = pd.DataFrame({'pred' : result})
result_df.to_csv('result.csv', index=False)
result_df

,pred
0,Poor
1,Good
2,Standard
3,Good
4,Standard
...,...
1494,Standard
1495,Poor
1496,Standard
1497,Good


In [43]:
test = pd.read_csv('result.csv')
test

,pred
0,Poor
1,Good
2,Standard
3,Good
4,Standard
...,...
1494,Standard
1495,Poor
1496,Standard
1497,Good
